## Import Libraries and Declare API Key

In [1]:
import requests
import pandas as pd
import numpy as np
import signal
import matplotlib.pyplot as plt
import talib
import time

In [2]:
API_KEY = {'X-API-key': 'RPCJTKLW'}

## Basic Buy and Sell Functions

In [3]:
shutdown = False

class ApiException(Exception):
    pass

def signal_handler(signum, frame):
    global shutdown
    signal.signal(signal.SIGINT, signal.SIG_DFL)
    shutdown = True

def get_tick(session):
    resp = session.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick']
    raise ApiException('Authorization error. Please check API key.')

def ticker_bid_ask(session, ticker):
    payload = {'ticker': ticker}
    resp = session.get('http://localhost:9999/v1/securities/book', params = payload)
    if resp.ok:
        book = resp.json()
        return book['bids'][0]['price'], book['asks'][0]['price']
    raise ApiException('Authorization error. Please check API key.')

def buy(session, ticker, type, q, price = None):
    if type == 'MARKET':
        resp = session.post('http://localhost:9999/v1/orders', params = {'ticker': ticker, 'type': 'MARKET', 'quantity': q, 'action': 'BUY'})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
    else:
        resp = session.post('http://localhost:9999/v1/orders',
                            params={'ticker': ticker, 'type': 'LIMIT', 'quantity': q, 'action': 'BUY', 'price': price})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')

def sell(session, ticker, type, q, price = None):
    if type == 'MARKET':
        resp = session.post('http://localhost:9999/v1/orders', params = {'ticker': ticker, 'type': 'MARKET', 'quantity': q, 'action': 'SELL'})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
    else:
        resp = session.post('http://localhost:9999/v1/orders',
                            params={'ticker': ticker, 'type': 'LIMIT', 'quantity': q, 'action': 'SELL', 'price': price})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')

def buy_mult(session, ticker, type, mult):
    for i in range(mult):
        buy(session, ticker, type, 10000)
        
def sell_mult(session, ticker, type, mult):
    for i in range(mult):
        sell(session, ticker, type, 10000)

def cancel(session, order_id):
    order_id = order_id # assuming the order to cancel has ID 100
    resp = s.delete('http://localhost:9999/v1/orders/{}'.format(order_id))
    if resp.ok:
        status = resp.json()
        success = status['success']
        print('The order was successfully cancelled?', success)
        

In [4]:
# with requests.Session() as s:
#     df_ngn = None
#     df_whel = None
#     df_gear = None
#     df_rsm = None

#     s.headers.update(API_KEY)
#     resp1 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'NGN'})
#     resp2 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'WHEL'})
#     resp3 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'GEAR'})
#     resp4 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'RSM1000'})
#     if resp1.ok and resp2.ok and resp3.ok and resp4.ok:
#         book1 = resp1.json()
#         book2 = resp2.json()
#         book3 = resp3.json()
#         book4 = resp4.json()
#         df_ngn = pd.DataFrame(book1)
#         df_whel = pd.DataFrame(book2)
#         df_gear = pd.DataFrame(book3)
#         df_rsm = pd.DataFrame(book4)
#     else:
#         print('request denied.')

## Algorithm Execution Section

In [ ]:
signal.signal(signal.SIGINT, signal_handler)
price = 10000

while True:
    with requests.Session() as s:
        df_ngn = None
        df_whel = None
        df_gear = None
        df_rsm = None
        
        s.headers.update(API_KEY)
        print(get_tick(s))
        resp1 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'NGN'})
        resp2 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'WHEL'})
        resp3 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'GEAR'})
        resp4 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'RSM1000'})
        
        if resp1.ok and resp2.ok and resp3.ok and resp4.ok:
            book1 = resp1.json()
            book2 = resp2.json()
            book3 = resp3.json()
            book4 = resp4.json()
            df_ngn = pd.DataFrame(book1)
            df_whel = pd.DataFrame(book2)
            df_gear = pd.DataFrame(book3)
            df_rsm = pd.DataFrame(book4)
        else:
            print('request denied.')
            
            
        _, _, macdhist_gear = talib.MACD(df_gear['close'].values, 10 , 20, 10)
        resp_gear_pos = s.get('http://localhost:9999/v1/securities', params = {'ticker' : 'GEAR'})
        gear_pos = resp_gear_pos.json()[0]['position']
        print(macdhist_gear)

              
        if macdhist_gear.size > 30:
            if macdhist_gear[28] < 0 and macdhist_gear[29] > 0:
                print('macd sell: ', macdhist_gear[28], macdhist_gear[29])
#                 if gear_pos > 0:
#                     sell_mult(s, 'GEAR','MARKET', int(gear_pos/10000))
                sell_mult(s, 'GEAR','MARKET', 10)
            if macdhist_gear[28] > 0 and macdhist_gear[29] < 0:
                print('macd buy: ', macdhist_gear[28], macdhist_gear[29])
#                 if gear_pos < 0:
#                     buy_mult(s, 'GEAR','MARKET', int(np.abs(gear_pos/10000)))
                buy_mult(s, 'GEAR','MARKET', 10)

                
        real_gr = talib.RSI(df_gear['close'].values, timeperiod=5)
    
        if real_gr.size > 6:
            if real_gr[5] > 40 and real_gr[6] < 40 and gear_pos > 0:
                print('rsi sell40: ', real_gr[5])
                sell_mult(s, 'GEAR','MARKET', int(gear_pos/10000))
                
            if real_gr[5] < 40 and real_gr[6] > 40 and gear_pos < 0:
                print('rsi buy40: ',real_gr[5])
                buy_mult(s, 'GEAR','MARKET', int(np.abs(gear_pos/10000)))
                
            if real_gr[5] < 60 and real_gr[6] > 60 and gear_pos > 0:
                print('rsi sell60: ',real_gr[5])
                sell_mult(s, 'GEAR','MARKET', int(gear_pos/10000))
                
            if real_gr[5] > 60 and real_gr[6] < 60 and gear_pos > 0:
                print('rsi buy60: ',real_gr[5])
                buy_mult(s, 'GEAR','MARKET', int(np.abs(gear_pos/10000)))

                
    time.sleep(1)

9
[nan nan nan nan nan nan nan nan nan]
10
[nan nan nan nan nan nan nan nan nan nan]
11
[nan nan nan nan nan nan nan nan nan nan nan]
12
[nan nan nan nan nan nan nan nan nan nan nan nan]
13
[nan nan nan nan nan nan nan nan nan nan nan nan nan]
14
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
15
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
16
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
17
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
18
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
19
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
20
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
21
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan]
22
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan]
23
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan

45
[        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan  0.10717917  0.09546455
  0.06476222  0.05186761  0.03621923  0.01762452 -0.00662728 -0.01650282
 -0.02338834 -0.02300786 -0.01959429 -0.02582568 -0.03176777 -0.0257357
 -0.01344896 -0.01225361 -0.02276942]
46
[        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan  0.11719275  0.11329143
  0.10088656  0.06960767  0.05622548  0.04015964  0.02120315 -0.00336575
 -0.0135

59
[        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan -0.07486875 -0.04989092
 -0.03416638 -0.01640349 -0.00836225  0.01811693  0.03991304  0.04678324
  0.0490029   0.04032314  0.04032391  0.03057915  0.05014459  0.06130979
  0.06296685  0.05573163  0.02921173  0.02017482  0.00805042 -0.0073501
 -0.02872492 -0.03602268 -0.04060841 -0.0381835  -0.0329579  -0.03758718
 -0.04211564 -0.0348383  -0.02145606 -0.01929795 -0.02896835]
60
[        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         

70
[        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan  0.06083139  0.06123712
  0.05745785  0.02923514 -0.00294255 -0.04473987 -0.06627992 -0.0669359
 -0.06673458 -0.05094025 -0.04280382 -0.03046132 -0.01056612  0.00067283
  0.01447746  0.01902559  0.04242162  0.06149564  0.06596161  0.06605678
  0.05549868  0.05383774  0.04262196  0.06088425  0.07089419  0.07152629
  0.06338101  0.03605244  0.02629637  0.01353191 -0.00243874 -0.02432179
 -0.03207293 -0.03706345 -0.03500019 -0.03009794 -0.03501653 -0.03980401
 -0.03275871 -0.01958448 -0.01761295 -0.0274508 ]
71
[        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         

81
[        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan -0.014527    0.00122451
  0.00556629 -0.00224518  0.00573739  0.01356879  0.01795158  0.02062846
  0.01920931  0.03729582  0.01567331  0.01023953  0.01562045  0.01654623
 -0.00729512 -0.03544147 -0.07356461 -0.09178163 -0.08945099 -0.08657926
 -0.06840751 -0.05816208 -0.04395427 -0.02241341 -0.00972554  0.00535261
  0.01101857  0.03539474  0.05532744  0.06054522  0.06129837  0.05131602
  0.05015885  0.03938388  0.05803199  0.06837973  0.06930773  0.0614218
  0.03432067  0.02476422  0.01217508 -0.00364145 -0.02538888 -0.0330206
 -0.03790584 -0.03574968 -0.03076538 -0.03561141 -0.04033467 -0.03323249
 -0.02000779 -0.01799147 -0.0277895 ]
macd sell:  

...

The market buy order was submitted and has ID 2131
The market buy order was submitted and has ID 2140
The market buy order was submitted and has ID 2141
The market buy order was submitted and has ID 2142
245
[            nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
 -1.10275359e-02  1.10014916e-02  1.23397819e-02 -1.09978555e-03
 -7.78522155e-03 -3.56950917e-03  6.29270942e-03  6.19866090e-03
 -1.35607741e-03 -2.10545684e-04 -8.74124496e-03 -2.39980365e-02
 -3.16906455e-02 -3.13963010e-02 -2.09873952e-03  1.88686651e-03
  7.29240606e-03  1.88701905e-02  1.28730001e-02  4.35109170e-03
  6.14365314

249
[            nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
  9.80678460e-02  5.59172815e-02  1.75238864e-02  7.08462832e-03
  9.23618649e-03  2.73530078e-02  2.56057960e-02  9.72909643e-03
  1.11475451e-03  3.79977807e-03  1.24432674e-02  1.13748907e-02
  3.03737032e-03  3.55035838e-03 -5.49489695e-03 -2.11734138e-02
 -2.92145256e-02 -2.92107353e-02 -1.57673472e-04  3.62022602e-03
  8.84763794e-03  2.02712491e-02  1.41394479e-02  5.49905403e-03
  7.18655653e-03  1.07528035e-02  1.02454696e-02  4.43626203e-03
 -4.24143710e-03 -6.31817440e-03 -2.83277213e-03 -4.33921522e-04
  1.95788794e-02  1.2

252
[            nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
  1.02462624e-01  1.12299045e-01  1.06631263e-01  8.09596058e-02
  4.16599040e-02  5.59668919e-03 -2.93325466e-03  7.87034187e-04
  2.01965976e-02  1.95181363e-02  4.52807505e-03 -3.34798670e-03
 -4.58135882e-05  9.11567906e-03  8.48393064e-03  5.16048161e-04
  1.34334511e-03 -7.43346023e-03 -2.28816800e-02 -3.07243631e-02
 -3.05488743e-02 -1.34663187e-03  2.56139458e-03  7.90273796e-03
  1.94264518e-02  1.33828917e-02  4.82051933e-03  6.57719993e-03
  1.02049403e-02  9.75239488e-03  3.99210378e-03 -4.64184023e-03
 -6.67937411e-03 -3.1

254
[            nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan             nan             nan
  2.55626435e-02  4.86458088e-02  7.83661089e-02  9.16170090e-02
  8.88322999e-02  6.56008980e-02  2.83717909e-02 -5.93000472e-03
 -1.29576357e-02 -7.95263748e-03  1.25584835e-02  1.28270700e-02
 -1.34660101e-03 -8.51703273e-03 -4.60336020e-03  5.08942702e-03
  4.92045401e-03 -2.64334497e-03 -1.46237332e-03 -9.92891467e-03
 -2.51043544e-02 -3.27067092e-02 -3.23190605e-02 -2.92917078e-03
  1.14511865e-03  6.63402476e-03  1.82889102e-02  1.23621227e-02
  3.90384633e-03  5.75344059e-03  9.46421099e-03  9.08594409e-03
  3.39216900e-03 -5.1

The market buy order was submitted and has ID 2249
The market buy order was submitted and has ID 2250


In [ ]:
# sell(s, 'GEAR','MARKET', 1000)

# with requests.Session() as s:
#     s.headers.update(API_KEY)
#     resp_gear_pos = s.get('http://localhost:9999/v1/securities', params = {'ticker' : 'GEAR'})
#     gear_pos = resp_gear_pos.json()[0]['position']
#     print(gear_pos)
    